In [1]:
!pip install mxnet                      > /dev/null
!pip install gluonnlp pandas tqdm       > /dev/null
!pip install sentencepiece              > /dev/null
!pip install transformers==3            > /dev/null
!pip install torch         

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master    > /dev/null

  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-mfotopas


In [3]:
import random
import numpy as np
import pandas as pd
import re

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp

from tqdm.notebook import tqdm

In [4]:
seed = 2021

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)  # Set random seed for all devices
torch.cuda.manual_seed_all(seed)  # Set random seed for all GPUs. If there is no GPU, it will be ignored by de

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
# GPU 사용 시
device = torch.device("cuda:0")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [9]:
df = pd.read_csv('데이터_추가2_전처리(불용어추가).csv')
df = df[['title', 'topic_idx']]
df.dropna(inplace=True)

train_df = df.sample(frac=0.8, random_state=2021)
val_df = df.drop(train_df.index)

train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

train_df.shape, val_df.shape

((42722, 2), (10681, 2))

In [21]:
dataset_val = []
for sen, label in zip(val_df['title'], val_df['topic_idx']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))

  dataset_val.append(data_train)
# kobert 트레인 할때 input 방식

In [19]:
dataset_train = []
for sen, label in zip(train_df['title'], train_df['topic_idx']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))

  dataset_train.append(data_train)

In [22]:
dataset_train[0], dataset_val[-1]

(['마트산업노동조합 손잡이 없는 명절 선물상자 공개', '2'], [' 朴대통령 내달 3일 방한 장관 이집트 대통령과 정상회의', '6'])

In [23]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [24]:
max_len = 64               
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3               
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [25]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [26]:
data_train = BERTDataset(dataset=dataset_train, sent_idx=0, label_idx=1, bert_tokenizer=tok, max_len=max_len, pad=True, pair=False)
data_val = BERTDataset(dataset=dataset_val, sent_idx=0, label_idx=1, bert_tokenizer=tok, max_len=max_len, pad=True, pair=False)

In [27]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [28]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [29]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [30]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [31]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [32]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [33]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [34]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [35]:
for e in range(num_epochs):
    train_acc = 0.0
    val_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train accuracy {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(val_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        val_acc += calc_accuracy(out, label)
    print("epoch {} validation accuracy {}".format(e+1, val_acc / (batch_id+1)))
    print()

  0%|          | 0/668 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.056990146636963 train acc 0.109375
epoch 1 batch id 201 loss 0.44498732686042786 train acc 0.6224347014925373
epoch 1 batch id 401 loss 0.40247294306755066 train acc 0.7487531172069826
epoch 1 batch id 601 loss 0.3328712582588196 train acc 0.7922472961730449
epoch 1 train accuracy 0.8022218430785488


  0%|          | 0/167 [00:00<?, ?it/s]

epoch 1 validation accuracy 0.8975371625170712



...

epoch 2 validation accuracy 0.9044608152116819



  0%|          | 0/668 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.2865450084209442 train acc 0.90625
epoch 3 batch id 201 loss 0.141643226146698 train acc 0.9370335820895522
epoch 3 batch id 401 loss 0.22404925525188446 train acc 0.9429161471321695
epoch 3 batch id 601 loss 0.17404931783676147 train acc 0.9467814059900166
epoch 3 train accuracy 0.9476309329869673


  0%|          | 0/167 [00:00<?, ?it/s]

epoch 3 validation accuracy 0.9061334567706691



In [36]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(sentence):
  data = [sentence, '0']
  dataset_another = [data]
  logits = 0
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
  
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model(token_ids, valid_length, segment_ids)

      for i in out:
        logits = i
        logits = logits.detach().cpu().numpy()
        logits = np.argmax(logits)

  return logits

using cached model


In [37]:
test_df = pd.read_csv('test_전처리.csv')
sub_df = pd.read_csv('sample_submission.csv')

In [ ]:
pre = []

for i in tqdm(test_df['title']):
  pre.append(predict(i))

  0%|          | 0/9131 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
sub_df['topic_idx'] = pre
sub_df.to_csv('데이터추가2_title컬럼.csv', index=False)